# Importing Libraries

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

# Load preprocessed data

In [3]:
X = np.load('data/preprocessed/features.npy')
Y = np.load('data/preprocessed/labels.npy')

# Split the data into training and testing set

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

# Normalize the data

In [5]:
x_train = x_train/x_train.max()
x_test = x_test/x_test.max()

# One hot encoding the labels

In [6]:
y_train

array([0., 6., 9., ..., 9., 9., 6.])

In [7]:
def one_hot_encode(arr:np.array):
    a = arr.max()
    one_hot = np.zeros((len(arr), a))
    final = []
    for i,j in zip(arr, one_hot):
        one_hot_row = np.insert(j, i, 1)
        final.append(one_hot_row)
    one_hot = np.array(final)
    return(one_hot)


In [8]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [9]:
y_train = one_hot_encode(y_train)
y_test = one_hot_encode(y_test)

# Building Neural Network

In [10]:
class DNN:
    def __init__(self, Learing_rate:float):
        ''' 
        Defining architeture of neural network

        784 neurons (input layer)
        392 neurons (sigmoid activation)
        392 neurons (sigmoid activation)
        186 neurons (sigmoid activation)
        10 neurons (output layer) (sigmoid activation)
        '''
        self.LearningRate = Learing_rate

        self.theta_matrix_1 = np.zeros((392, 785))
        self.theta_matrix_2 = np.zeros((392, 393))
        self.theta_matrix_3 = np.zeros((186, 393))
        self.theta_matrix_4 = np.zeros((10, 187))

        self.delta_matrix_4 = np.zeros((10, 187))
        self.delta_matrix_3 = np.zeros((186, 393))
        self.delta_matrix_2 = np.zeros((392, 393))
        self.delta_matrix_1 = np.zeros((392, 785))
    
    def g(self, inp):
        return(1/(1+np.exp(-inp)))
    
    def forwardpropogation(self, X:np.array):
        global a1,a2,a3,a4,a5
        a1 = X.T
        a1 = np.insert(a1, 0, 1)

        z2 = np.matmul(self.theta_matrix_1, a1)
        a2 = self.g(z2)
        a2 = np.insert(a2, 0, 1)

        z3 = np.matmul(self.theta_matrix_2, a2)
        a3 = self.g(z3)
        a3 = np.insert(a3, 0, 1)

        z4 = np.matmul(self.theta_matrix_3, a3)
        a4 = self.g(z4)
        a4 = np.insert(a4, 0, 1)

        z5 = np.matmul(self.theta_matrix_4, a4)
        a5 = self.g(z5)
        hx = a5
        return(hx)
    
    def backpropogation(self, output_layer:np.array, Y:np.array):
        delta_5 = output_layer - Y.T
        delta_4 = np.matmul(self.theta_matrix_4.T, delta_5)
        delta_3 = np.matmul(self.theta_matrix_3.T, delta_4[1:])
        delta_2 = np.matmul(self.theta_matrix_2.T, delta_3[1:])

        self.delta_matrix_4 += self.delta_matrix(delta_5, a4)
        self.delta_matrix_3 += self.delta_matrix(delta_4[1:], a3)
        self.delta_matrix_2 += self.delta_matrix(delta_3[1:], a2)
        self.delta_matrix_1 += self.delta_matrix(delta_2[1:], a1)

    def delta_matrix(self, delta_vector:np.array, activation_layer:np.array):
        final_delta_matrix = []
        for i in delta_vector:
            a = i*activation_layer
            final_delta_matrix.append(a.T)
        return(np.array(final_delta_matrix))

    def GradientDescent(self, theta_matrix:np.array, delta_matrix:np.array, m:int, lamb):
        new_delta = (1/m)*delta_matrix

        D_bias = new_delta[:, 0]
        D_weights = new_delta[:, 1:] + theta_matrix[:, 1:]*lamb

        new_bias_theta = theta_matrix[:, 0] - self.LearningRate*D_bias
        new_weights_theta = theta_matrix[:, 1:] - self.LearningRate*D_weights

        new_bias_theta.shape = (new_bias_theta.shape[0], 1)

        new_theta_matrix = np.concatenate((new_bias_theta, new_weights_theta), axis=1)
        return(new_theta_matrix)
    
    def fit(self, x_train:np.array, y_train:np.array, epochs = 10):
        m = len(x_train)

        for k in range(epochs):
            for i,j in zip(x_train, y_train):
                hx = self.forwardpropogation(i)
                self.backpropogation(hx, j)
            t1 = self.GradientDescent(self.theta_matrix_1, self.delta_matrix_1, m, 0.1)
            t2 = self.GradientDescent(self.theta_matrix_2, self.delta_matrix_2, m, 0.1)
            t3 = self.GradientDescent(self.theta_matrix_3, self.delta_matrix_3, m, 0.1)
            t4 = self.GradientDescent(self.theta_matrix_4, self.delta_matrix_4, m, 0.1)

            self.theta_matrix_1 = t1
            self.theta_matrix_2 = t2
            self.theta_matrix_3 = t3
            self.theta_matrix_4 = t4
            print(f"epoch : {k}")

    def accuracy(self, hx:np.array, y_test:np.array):
        acc = sum(np.mod(hx-y_test.T)/y_test.T)/len(y_test)
        return(acc)
    
    def predict(self, x_test:np.array, y_test:np.array):
        predict_list = []
        for i,j in zip(x_test, y_test):
            actual = np.argmax(j)
            hx = self.forwardpropogation(i)
            accuracy = self.accuracy(hx, j)
            value = np.argmax(hx)
            predict_list.append((hx, value))
        return(predict_list)
    
    def save(self, path:str):
        theta_tensor = np.array([self.theta_matrix_1, self.theta_matrix_2, self.theta_matrix_3, self.theta_matrix_4])
        np.save(path, theta_tensor)

# Training

In [11]:
model = DNN(0.01)

In [12]:
model.fit(x_train, y_train, epochs=10)

epoch : 0
epoch : 1
epoch : 2
epoch : 3
epoch : 4
epoch : 5
epoch : 6
epoch : 7
epoch : 8
epoch : 9


In [13]:
predictions = model.predict(x_test, y_test)

TypeError: remainder() takes from 2 to 3 positional arguments but 1 were given